#Setup

In [1]:
#pip install keras-adabound

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
#from keras_adabound import AdaBound
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.metrics import mean_squared_error as calc_mse
import time

In [3]:
csv = pd.read_csv('https://raw.githubusercontent.com/clement880101/MLStocks/master/Msft.csv', date_parser= True)
csv.columns

Index(['date', 'Msft_open', 'Msft_high', 'Msft_low', 'Msft_close',
       'Msft_adjusted_close', 'Msft_volume', 'Msft_dividend',
       'Msft_split_coefficent', 'Msft_Real Middle Band',
       'Msft_Real Upper Band', 'Msft_Real Lower Band', 'Msft_MACD',
       'Msft_MACD_Hist', 'Msft_MACD_Signal', 'Msft_RSI', 'Msft_CCI',
       'Msft_Aroon Down', 'Msft_Aroon Up', 'Msft_Chaikin A/D', 'Msft_ADX',
       'Msft_SMA', 'SPY_open', 'SPY_high', 'SPY_low', 'SPY_close',
       'SPY_adjusted_close', 'SPY_volume', 'SPY_dividend',
       'SPY_split_coefficent'],
      dtype='object')

# Recurrent LSTM Neural Network

###help functions / RNN Class

In [4]:
#helper functions
def to_dataframe(csv):
    # returns dataframe
    df = pd.read_csv(csv, date_parser=True)
    return df

def reverse_order(df):
    # reverse order of data so earliest day is day 0
    reversed_df = df[::-1].reset_index(drop=True)
    return reversed_df

def remove_dates(df):
  #stores dates in a dictionary
  dates = {}
  for i in range(df.shape[0]):
    date = df.iloc[i]['date']
    dates[date] = i

  df = df.drop(['date'],axis=1)

  return dates, df

def scale_data(df, target_column):
    # scale data
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df)

    # save this value to convert stock prediction to nominal data
    upscale_value = 1 / scaler.scale_[target_column]
    return scaled_data, scaler, upscale_value

def split_data(df, date_value):
  df_before = df[df['date'] < date_value].copy()
  df_after = df[df['date'] >= date_value].copy()
  return df_before, df_after

def create_xy(data, scope, target_column):
  x = []
  y = []
  for i in range(scope, data.shape[0]):
    # the xTest will have an array of the last x "scope" days of data
    # yTest will be the the opening value of the next day
    x.append(data[i-scope:i])
    y.append(data[i,target_column])
  
  #the length of x is the data length - scope 
  #in each x there is a batch size of x "scope" points
  return np.array(x), np.array(y)
#recurrent neural network

class Rnn:
    # set values for Rnn object
    def __init__(self, rows_size, columns_size):
        self.rows = rows_size
        self.columns = columns_size
        self.model = None
        self.logs = None


    # train function for Rnn class
    def structure(self, layers, units_for_layers, dropouts_for_layers):
        #initilize Sequential rnn
        self.model = Sequential()
        # add first layer and define input shape
        self.model.add(LSTM(units_for_layers[0], activation = 'relu', return_sequences = True, input_shape = (self.rows, self.columns)))
        self.model.add(Dropout(dropouts_for_layers[0]))
        # for adding additional layers
        if layers > 2:
            for i in range(1,layers):

                return_setting = True
                #dont need to return values upstream on last layer
                if i == layers - 1: return_setting = False

                self.model.add(LSTM(units_for_layers[i], activation = 'relu', return_sequences = return_setting))
                self.model.add(Dropout(dropouts_for_layers[i]))


        #final endpoint for rnn layers
        self.model.add(Dense(units = 1))
        return None

    def summary(self):
        return self.model.summary()
      
    def history(self):
      return self.model.history

    def train(self, xTrain, yTrain, epochs, batch_size, optimizer, file_name, ada_low_lr = None, ada_high_lr = None):
        #compiles model that was created
        if optimizer != 'adaboost':
            self.model.compile(optimizer=optimizer, loss = 'mean_squared_error')
        else:
            self.model.compile(optimizer= AdaBound(lr=ada_low_lr, final_lr=ada_high_lr), loss = 'mean_squared_error')

        #create log for getting stats
        CSV_logger = CSVLogger(file_name + '.csv',separator=',',append=False)
        #fit model to data
        self.model.fit(xTrain, yTrain, epochs=epochs, batch_size=batch_size, callbacks=[CSV_logger])
        self.logs = CSV_logger
        return None

    def predict(self, input_data):
        y_hat = self.model.predict(input_data)
        return y_hat

In [5]:
def create_file_name(array):
  string = 'Stats'
  for item in array:
    string = string + '_' + str(item)
  return string

###main

In [6]:
#arrange df and split by date
df = reverse_order(csv)
training_data, test_data = split_data(df, '2018-01-01')

#store date labels and drop columns 
dates_train, training_data = remove_dates(training_data)
dates_test, test_data = remove_dates(test_data)

target_column = 5 #this is MSFT adjusted column
#scale_data on training data and get scaler with value
training_data, scaler, upscale_value = scale_data(training_data, target_column-1)
test_data = scaler.transform(test_data)


In [7]:
#create xTrain,yTrain.. and xTest,yTest
#each x in xTrain will be an array of x days
xTrain, yTrain = create_xy(training_data, 5, target_column-1)
xTest, yTest = create_xy(test_data, 5, target_column-1)

###Params loop

In [8]:
#batch dimensions
row_size = xTrain.shape[1]
column_size = xTrain.shape[2]
# params to test
layers = [3,5]
epochs = [10, 25, 50]
batches = [100, 200]
optimizer = 'adam'

In [9]:
params = []
csv_name = 'MSFT'

for l in layers:
  units = []
  dropouts = []
  for i in range(l):
    units.append(column_size)
    if i == 0:
      dropouts.append(0.2)
    else: 
      dropouts.append(0.5)

  for e in epochs:
    for b in batches:
      file_name = create_file_name([csv_name,l,e,b])
      params.append({'layers': l,'epochs': e,'batches': b, 'units': units, 'dropouts': dropouts, 'optimizer': optimizer, 'file_name': file_name})

In [10]:
params[0]['file_name']

'Stats_MSFT_3_10_100'

### Create evaluation loop

In [11]:
#track_time_to_train
time_training = []
networks = []
for param in params:
  t0 = time.clock()
  #Steps for rnn:
  #1. initialize, #2. structure, #train, #summary, #predict
  print('Starting this param'), print(param)
  nnet = Rnn(row_size,column_size)
  nnet.structure(param['layers'], param['units'], param['dropouts'])
  nnet.train(xTrain, yTrain, param['epochs'],param['batches'],param['optimizer'],param['file_name'])
  networks.append(nnet)
  t1 = time.clock()
  time_training.append(t1-t0)


Starting this param
{'layers': 3, 'epochs': 10, 'batches': 100, 'units': [29, 29, 29], 'dropouts': [0.2, 0.5, 0.5], 'optimizer': 'adam', 'file_name': 'Stats_MSFT_3_10_100'}
Epoch 1/10
46/46 [==============================] - 1s 11ms/step - loss: 0.0352
Epoch 2/10
46/46 [==============================] - 0s 10ms/step - loss: 0.0108
Epoch 3/10
46/46 [==============================] - 1s 11ms/step - loss: 0.0083
Epoch 4/10
46/46 [==============================] - 1s 12ms/step - loss: 0.0074
Epoch 5/10
46/46 [==============================] - 0s 11ms/step - loss: 0.0070
Epoch 6/10
46/46 [==============================] - 0s 11ms/step - loss: 0.0066
Epoch 7/10
46/46 [==============================] - 1s 11ms/step - loss: 0.0061
Epoch 8/10
46/46 [==============================] - 1s 11ms/step - loss: 0.0064
Epoch 9/10
46/46 [==============================] - 1s 11ms/step - loss: 0.0051
Epoch 10/10
46/46 [==============================] - 1s 11ms/step - loss: 0.0052
Starting this param
{'laye

###Testing

Testing Loop

In [12]:
def save_test_image(yTest, y_pred, file_name):
  plt.plot(yTest, color='black', label='Actual')
  plt.plot(y_pred, color='red', label='Predict')
  plt.title('Testing')
  plt.xlabel('time [days]')
  plt.ylabel('price')
  plt.legend(loc='best')
  plt.savefig(fname = (file_name +'_test_img.jpg'))
  plt.close()
  return None

In [13]:
def save_train_image(yTrain, y_pred, file_name):
  plt.plot(yTrain, color='black', label='Actual')
  plt.plot(y_pred, color='red', label='Predict')
  plt.title('Training')
  plt.xlabel('time [days]')
  plt.ylabel('price')
  plt.legend(loc='best')
  plt.savefig(fname = (file_name+'_train_img.jpg'))
  plt.close()
  return None

In [14]:
time_training

[11.781369000000002,
 9.013605000000002,
 24.04926,
 17.516650000000006,
 45.746174999999994,
 32.02353599999999,
 18.646477000000004,
 14.909222999999997,
 39.22663699999998,
 28.986884000000003,
 72.069167,
 51.97744399999999]

In [15]:
time_evaluating = []
header = ['file_name: layers, epoch, batch', 'mse_scaled', 'mse_unscaled', 'overall_time']
results = []
yTest_scaled = yTest * upscale_value
yTrain_scaled = yTrain * upscale_value

for i in range(len(networks)):
  t0 = time.clock()
  nnet = networks[i]
  file_name = params[i]['file_name']
  #test images and result
  yPred_test = nnet.predict(xTest)
  yPred_test_scaled = yPred_test * upscale_value
  save_test_image(yTest_scaled, yPred_test_scaled, file_name)

  mse_unscaled = calc_mse(yTest, yPred_test)
  mse_scaled = calc_mse(yTest_scaled, yPred_test_scaled)

  #training images
  yPred_train = nnet.predict(xTrain)
  yPred_train_scaled = yPred_train * upscale_value
  save_train_image(yTrain_scaled, yPred_train_scaled, file_name)
  t1 = time.clock()

  time_evaluating = t1-t0
  overall_time = time_training[i] + time_evaluating
  stats = [file_name, round(mse_scaled,3), round(mse_unscaled,3), round(overall_time, 3)]
  results.append(stats)

In [16]:
results_df = pd.DataFrame(data = results, columns= header)
results_df.to_csv('LSTM_' + csv_name+'_results.csv')

In [17]:
!zip -r /content/file.zip /content/

from google.colab import files
files.download("/content/file.zip")

  adding: content/ (stored 0%)
  adding: content/.config/ (stored 0%)
  adding: content/.config/.last_survey_prompt.yaml (stored 0%)
  adding: content/.config/config_sentinel (stored 0%)
  adding: content/.config/.last_update_check.json (deflated 22%)
  adding: content/.config/active_config (stored 0%)
  adding: content/.config/.last_opt_in_prompt.yaml (stored 0%)
  adding: content/.config/gce (stored 0%)
  adding: content/.config/logs/ (stored 0%)
  adding: content/.config/logs/2020.12.02/ (stored 0%)
  adding: content/.config/logs/2020.12.02/22.03.37.873126.log (deflated 91%)
  adding: content/.config/logs/2020.12.02/22.04.13.854338.log (deflated 87%)
  adding: content/.config/logs/2020.12.02/22.04.21.823807.log (deflated 54%)
  adding: content/.config/logs/2020.12.02/22.03.59.234441.log (deflated 53%)
  adding: content/.config/logs/2020.12.02/22.04.37.441505.log (deflated 54%)
  adding: content/.config/logs/2020.12.02/22.04.38.150307.log (deflated 54%)
  adding: content/.config/conf

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>